## 22단계: 연산자 오버로드(3)

> 이전 단계에서는 DeZero가 *와 + 연산자를 지원하도록 확장했는데, 연산자는 이밖에도 많습니다. \
그래서 이번 단계에서는 [표 22-1]의 연산자들을 새로 추가하겠습니다.

<img src="images/표 22-1.png" width=350/>

첫 번째 메서드인 `__neg__`는 부호 변환 연산자이며, '단항 연산자'입니다. 그래서 `other`을 받지 않습니다. 나머지 연산자들은 차례로 뺄셈, 나눗셈, 거듭제곱으로, 모두 이항 연산자입니다(a - b, a / b 등). 따라서 적용 대상이 우항이냐 좌항이냐에 따라 2개의 특수 메서드 중 하나가 선별되어 호출됩니다. 단, 거듭제곱은 `x ** 3`처럼 좌항이 Variable 인스턴스이고 우항이 상수(2, 3등의 int)인 경우만을 고려하겠습니다.

In [1]:
# 필요 모듈 정의

import weakref
import contextlib
from heapq import heappush, heappop
import numpy as np


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200
    
    def __init__(self, data, name=None):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return 'Variable(None)'
        p = str(self.data).replace('\n', '\n' + ' '*9)
        return f'Variable({p})'
    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def ndim(self):
        return self.data.ndim
    
    @property
    def size(self):
        return self.data.size
    
    @property
    def dtype(self):
        return self.data.dtype
    
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1
    
    def cleargrad(self):
        self.grad = None
    
    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs_heap = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                heappush(funcs_heap, f)
                seen_set.add(f)
        
        add_func(self.creator)
        
        while funcs_heap:
            f = heappop(funcs_heap)
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                
                if x.creator is not None:
                    add_func(x.creator)
            
            if not retain_grad:
                for y in f.outputs:
                    y.grad = None


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]
        
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])  # Todo: __lt__와 충돌 가능
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.proxy(output) for output in outputs]
        
        return outputs if len(outputs) > 1 else outputs[0]

    def __lt__(self, other):
        return self.generation > other.generation
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


# class Square(Function):
#     def forward(self, x):
#         y = x ** 2
#         return y
    
#     def backward(self, gy):
#         x = self.inputs[0].data
#         gx = 2 * x * gy
#         return gx


# def square(x):
#     return Square()(x)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

### 22.1 음수(부호 변환)

음수(부호 변환)의 미분은 $y = -x$일 때 $\frac{\partial y}{\partial x} = -1$이다.

In [2]:
class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy

def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

x = Variable(np.array(2.0))
y = -x
y.backward()
print(y)
print(x.grad)

Variable(-2.0)
-1.0


### 22.2 뺄셈

뺄셈의 미분은 $y = x_0 - x_1$일 때 $\frac{\partial y}{\partial x_0} = 1$, $\frac{\partial y}{\partial x_1} = -1$이다.

In [3]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

그러나 `__rsub__`의 경우, 부호가 반대가 되는 점 때문에 전용 함수를 만들어줘야 한다.

(공식 GitHub 레포에 그림 업로드 안 됨)

In [4]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0)

Variable.__rsub__ = rsub


x = Variable(np.array(2.0))
y1 = 2.0 - x
y2 = x - 1.0
y = y1 + y2
y.backward()

print(y1)
print(y2)
print(x.grad)

Variable(0.0)
Variable(1.0)
0.0


### 22.3 나눗셈

나눗셈의 미분은 $y = x_0 / x1$일 때 $\frac{\partial y}{\partial x_0} = 1 / x_1$, $\frac{\partial y}{\partial x_1} = - x_0 / (x_1)^2$이다. \
나눗셈도 뺄셈과 마찬가지로 좌항, 우항의 순서에 주의하자.

In [5]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0)

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
"""왜 __div__가 아닌 __truediv__ 일까?

__div__는 파이썬 2.x에서 사용되던 나눗셈 연산자이고, __truediv__는 파이썬 3.x에서 사용되는 나눗셈 연산자이다.
>>> 파이썬 2.x: 정수끼리는 정수 나눗셈, 하나라도 실수라면 실수 나눗셈
>>> 파이썬 3.x: 항상 실수 나눗셈 (정수 나눗셈은 // 연산자 사용)
"""

x = Variable(np.array(2.0))
y1 = 2.0 / x
y2 = x / 1.0
y = y1 + y2
y.backward()

print(y1)
print(y2)
print(x.grad)

Variable(1.0)
Variable(2.0)
0.5


### 22.4 거듭제곱

거듭제곱의 미분은 $y = x^c$일 때 $\frac{\partial y}{\partial x} = cx^{c-1}$이 된다.

In [6]:
class Pow(Function):
    def __init__(self, c):
        self.c = c
        
    def forward(self, x):
        y = x ** self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx
    

def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow


x = Variable(np.array(2.0))
y = x ** 3
y.backward()

print(y)
print(x.grad)

Variable(8.0)
12.0


Q. PyTorch는 거듭제곱을 어떻게 구현했을까?

|        상황       |           PyTorch 동작           |
|:-----------------:|:--------------------------------:|
| x > 0, 실수 지수  | OK, 정상 작동                    |
| x < 0, 정수 지수  | OK, 예: (-2) ** 3 = -8           |
| x < 0, 실수 지수  | NaN 발생, 정의되지 않음          |
| x == 0, 양 지수   | 0                                |
| x == 0, 음 지수   | inf 또는 NaN (0으로 나누기 발생) |
| x == 0, 지수 == 0 | 1.0 (정의상 관례)                |